# API Management operation statistics

With this notebook API Management all defined operations are fetched and matched with their request counts fetched from Application Insights.

## Requirements
- API Management integration to Application Insights
- environment variables set to your values (see _environment variables_)
- Service Principal in AAD with (see _creating a Service Principal for managing ARM resources_)
- adal package installed into current jupyter kernel (see _Install the adap pip packagel in the current Jupyter kernel_)

### environment variables
These environment variables need to be set with correct values 

| name | purpose | source |
| --- | --- | --- |
| AzMgmtAppId | AppId in AAD to allow access to ARM resources in your subscription | see 'creating a Service Principal for managing ARM resources' |
| AzMgmtPassword | Password for AppId above | see 'creating a Service Principal for managing ARM resources' |
| AzMgmtTenant | Tentant Id of AAD used to manage Azure subscription  | Azure Portal; AAD used to manage Azure subscription |
| AzMgmtSubscription | Id of subscription containing Api Management and Application Insights | Azure Portal |
| AzApiMgmtResourceGroup | Resource Group containing Api Management | Azure Portal |
| AzApiMgmtServiceName | API Management Service Name | Azure Portal |
| AzAppInsightsAppId | Application Insights AppId | Azure Portal |
| AzAppInsightsApiKey | Application Insights ApiKey used for REST API access | Azure Portal |

This can be achieved with a PowerShell setting the environment variables for the current session and for the user persistently:
```
Write-Host "setting Azure Environment secrets"
$env:AzMgmtAppId = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtAppId",$env:AzMgmtAppId,"User")

$env:AzMgmtPassword = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtPassword",$env:AzMgmtPassword,"User")

$env:AzMgmtTenant = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtTenant",$env:AzMgmtTenant,"User")

$env:AzMgmtSubscription = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtSubscription",$env:AzMgmtSubscription,"User")

$env:AzApiMgmtResourceGroup = "{your-value}"
[Environment]::SetEnvironmentVariable("AzApiMgmtResourceGroup",$env:AzApiMgmtResourceGroup,"User")

$env:AzApiMgmtServiceName = "{your-value}"
[Environment]::SetEnvironmentVariable("AzApiMgmtServiceName",$env:AzApiMgmtServiceName,"User")

$env:AzAppInsightsAppId = "{your-value}"
[Environment]::SetEnvironmentVariable("AzAppInsightsAppId",$env:AzAppInsightsAppId,"User")

$env:AzAppInsightsApiKey = "{your-value}"
[Environment]::SetEnvironmentVariable("AzAppInsightsApiKey",$env:AzAppInsightsApiKey,"User")
```

### creating a Service Principal for managing ARM resources
To create a Service Principal with PowerShell login to your AAD/Azure subscription with AzureRM, switch to the target subscription (containing API Management and Application Insights resources) and execute these steps:

```
$sp = New-AzureRmADServicePrincipal -DisplayName myAzureManagement -Password "thepassword"
New-AzureRmRoleAssignment -RoleDefinitionName Contributor -ServicePrincipalName $sp.ApplicationId
```

Note ```ApplicationId``` and put it into the environment variables.

### Install the adap pip packagel in the current Jupyter kernel
To install adal package into your kernel, copy these code lines into a newly inserted cell and execute.

```python
import sys
!{sys.executable} -m pip install adal
``` 

In [ ]:
# obtain authorization token to be used in API Management REST API
import adal
import os
import json

def getToken():
    AppId = os.environ.get('AzMgmtAppId')
    AppPassword = os.environ.get('AzMgmtPassword')
    tenantID = os.environ.get('AzMgmtTenant')
    authURL = "https://login.windows.net/" + tenantID
    resource = "https://management.azure.com/"

    context = adal.AuthenticationContext(
        authURL, validate_authority=tenantID, api_version=None)

    token = context.acquire_token_with_client_credentials(
        resource,
        AppId,
        AppPassword)

    return token

def getAuthorizationHeader():
    token = getToken()

    header = token['tokenType'] + ' ' + token['accessToken']

    return header


# ----------------------------------------------------------------------

authHeader = getAuthorizationHeader()

print(authHeader)


In [ ]:
# use API Management REST API to obtain APIs
import requests

resource = "https://management.azure.com"
resourceGroupName = os.environ.get('AzApiMgmtResourceGroup')
serviceName = os.environ.get('AzApiMgmtServiceName')
subscriptionId = os.environ.get('AzMgmtSubscription')
apiVersionSuffix = "?api-version=2017-03-01"
        
url = '{}/subscriptions/{}/resourceGroups/{}/providers/Microsoft.ApiManagement/service/{}/apis{}'.format(resource,subscriptionId,resourceGroupName,serviceName,apiVersionSuffix)
headers = {"Content-Type": "application/json","Authorization":authHeader}

response = requests.get(url,headers=headers)


In [ ]:
print(response.json()['value'][:2])

In [ ]:
# transform API result into dataframe
import pandas as pd
apis = pd.DataFrame.from_dict(response.json()['value'])
apis.head(n=2)

In [ ]:
# pull path from properties into data frame
path = apis.properties.apply(lambda x : x["path"])
apis = apis.assign(path=path)
apis.head(n=2)


In [ ]:
# create a data frame with all operations in all APIs
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# get operations for a single API
def getOperations(id):
    url = '{}{}/operations{}'.format(resource,urllib.parse.quote(id),apiVersionSuffix)
    print(url)
    headers = {"Content-Type": "application/json","Authorization":authHeader}
    response = requests_retry_session().get(url,headers=headers)
    print(response.status_code)
    return response.json()["value"]

# loop thru all APIs, fetch operations
for index, api in apis.iterrows():
    print(index,api.id)
    apiOperations = pd.DataFrame.from_dict(getOperations(api.id))
    apiOperations['path'] = api.path
    operation = apiOperations.properties.apply(lambda x : x["method"] + " /" + api.path + x["urlTemplate"])
    apiOperations = apiOperations.assign(operation=operation)
    apiOperations['counter'] = 0
    if index == 0:
        operations = apiOperations
    else:
        operations = operations.append(apiOperations,ignore_index=True)
    
operations.head(n=2)

In [ ]:
import requests
import urllib
import pandas as pd

appInsightsAppId = os.environ.get('AzAppInsightsAppId')
appInsightsApiKey = os.environ.get('AzAppInsightsApiKey')
appInsightsQuery = 'requests | where timestamp >= ago(30d) and tostring(customDimensions.ApimanagementServiceName) != "" and name !startswith "OPTIONS" | summarize calls=count() by name'
appInsightsUrl = "https://api.applicationinsights.io/v1/apps/{}".format(appInsightsAppId)

url = '{}/query?query={}'.format(appInsightsUrl,urllib.parse.quote(appInsightsQuery))
headers = {"Content-Type": "application/json","x-api-key":appInsightsApiKey}

response = requests.get(url,headers=headers)

stats = pd.DataFrame.from_dict(response.json()["tables"][0]["rows"])
stats.head(n=2)

In [ ]:
# match API Operations and Statistics
import re

operationResults = pd.DataFrame(columns=['api','operation','counter'])

for opIndex, op in operations.iterrows():
    # replace {url-path-element} by wildcard
    operationMatch = re.sub('\{(\w+)\}', '.*',op.operation)
    # remove trailing /
    operationMatch = re.sub('\/$','',operationMatch)
    # remove URL parameters after &
    operationMatch = operationMatch.split('?')[0]
    regexMatch = re.compile(operationMatch,re.I)

    counter = 0
    statIndexList = []
    for statIndex, stat in stats.iterrows():
        if regexMatch.match(stat[0]):
            counter += stat[1]
            statIndexList.append(statIndex)
    
    # delete all matched statistics
    stats = stats.drop(statIndexList)
    
    operationResult = {'api':op.path,'operation':op.operation,'counter':counter}
    print(operationResult)
    operationResults.loc[opIndex] = operationResult
    


            


In [ ]:
operationResults.sort_values(by='counter', ascending=False)

In [ ]:
# there should be nothing left in stats - otherwise this indicates a matching problem

pd.set_option('display.max_colwidth', -1)
stats